In [1]:
#importing libraries
import numpy as np
import torch
import pandas as pd
import sklearn  
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from numba import njit, prange

In [2]:
#load dataset 
dataset = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv")

In [3]:
#dataset
dataset.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [4]:
#X Y split of the data
X = dataset.iloc[:,0:8].values        #.value convert the dataset type to numpy
Y = dataset.iloc[:,8].values

#converting X Y into tensors
X = torch.tensor(X, dtype= torch.float32)
Y = torch.tensor(Y, dtype= torch.float32)

In [5]:
print(X.shape, Y.shape)

torch.Size([767, 8]) torch.Size([767])


In [6]:
###Test train split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state = 42)


In [7]:
print(X_train.shape , X_test.shape, Y_train.shape, Y_test.shape, type(X_train))

torch.Size([613, 8]) torch.Size([154, 8]) torch.Size([613]) torch.Size([154]) <class 'torch.Tensor'>


In [8]:
#model definition
class NN_model(nn.Module):
    def __init__(self, input_features=8, hidden_layer1 = 20, hidden_layer2 = 20, output_features=1):
        super().__init__()
        self.from_input_to_H1 = nn.Linear(input_features, hidden_layer1)
        self.from_H1_to_H2 = nn.Linear(hidden_layer1,hidden_layer2)
        self.from_H2_to_output = nn.Linear(hidden_layer2,output_features)
        #self.relu = nn.ReLU()
    def forward(self,x):
        x = F.relu(self.from_input_to_H1(x))
        x = F.relu(self.from_H1_to_H2(x))
        x = F.sigmoid(self.from_H2_to_output(x))
        return x
        

In [9]:
torch.manual_seed(20)
 

In [10]:
#model creating
model1 = NN_model()
loss_fn1 = nn.BCELoss()
optimizer1 = optim.Adam(model1.parameters(),lr=0.0001)
model2= NN_model()
loss_fn2 = nn.BCELoss()
optimizer2 = optim.Adam(model2.parameters(),lr=0.00001)

In [37]:

def model_fn(X_train, Y_train, model, optimizer, loss_fn):
    epoch = 100000
    batch_size = 10
    loss_calc=[]
    for i in range (epoch):
        epoch_loss = 0
        for a in range(0, len(X_train), batch_size):
            X_batch = X_train[a:a+batch_size]
            optimizer.zero_grad()
            Y_pred = model(X_batch)
            Y_batch = Y_train[a:a+batch_size]
            loss = loss_fn(Y_pred.squeeze(), Y_batch)
            loss.backward()
            optimizer.step()
            # print(loss)
            epoch_loss = epoch_loss + loss.item()
            
        epoch_loss = epoch_loss/(len(X_train)/batch_size)
        loss_calc.append(epoch_loss)

        if i % 100 ==0:
            print("Epoch number",i)
            print(epoch_loss)

    plt.plot(loss_calc)
    plt.legend()
    plt.show()

In [38]:
model_fn(X_train,Y_train, model1, optimizer1,loss_fn1)

Epoch number 0
0.8763702021726387
Epoch number 100
0.5826957511668494
Epoch number 200
0.5531472385804767
Epoch number 300
0.5342008209928603
Epoch number 400
0.5200832805081255
Epoch number 500
0.5071725104411313
Epoch number 600
0.4951756441476489
Epoch number 700
0.484770184434452
Epoch number 800
0.4753856166443662
Epoch number 900
0.4664872778764946
Epoch number 1000
0.4586134583584442
Epoch number 1100
0.4515002412663782
Epoch number 1200
0.4448052944211353
Epoch number 1300
0.438544477714411
Epoch number 1400
0.4326547864599679
Epoch number 1500
0.4270417667427032
Epoch number 1600
0.42228495437791647
Epoch number 1700
0.4178293319160748
Epoch number 1800
0.4133506323443346
Epoch number 1900
0.4086164245305987
Epoch number 2000
0.4042108525897317
Epoch number 2100
0.4008377875774175
Epoch number 2200
0.3972539773627168
Epoch number 2300
0.3944763335814095
Epoch number 2400
0.3912809860647989
Epoch number 2500
0.3885260354393264
Epoch number 2600
0.3860325655530755
Epoch number 2

KeyboardInterrupt: 

In [11]:
model = NN_model()
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)
# def apply_model(x):
#     return model(x)

In [ ]:
%%time
from torch.func import vmap
epoch = 100000
batch_size = 10
loss_calc=[]
for i in range (epoch):
    epoch_loss = 0
    for a in range(0, len(X_train), batch_size):
        X_batch = X_train[a:a+batch_size]
        optimizer.zero_grad()
        Y_pred = vmap(apply_model)(X_batch)
        break

In [35]:
Y_pred

tensor([[0.7687],
        [0.9985],
        [0.6726],
        [0.9684],
        [0.5449],
        [0.6146],
        [0.6787],
        [0.9873],
        [0.9952],
        [0.9862]], grad_fn=<SigmoidBackward0>)

In [36]:
%%time
from torch.func import vmap
epoch = 100000
batch_size = 10
loss_calc=[]
for i in prange (epoch):
    epoch_loss = 0
    for a in prange(0, len(X_train), batch_size):
        X_batch = X_train[a:a+batch_size]
        optimizer.zero_grad()
        Y_pred = model(X_batch)
        break

CPU times: user 9.32 s, sys: 190 ms, total: 9.51 s
Wall time: 9.51 s


In [14]:
import torch
from torch import vmap
from numba import prange

def model_fn(X_train, Y_train, model, optimizer, loss_fn):
    epoch = 1000
    batch_size = 159
    loss_calc = []

    # Define a function that applies the model and calculates the loss for a single batch
    def compute_loss(X_batch, Y_batch):
        Y_pred = model(X_batch)
        loss = loss_fn(Y_pred.squeeze(), Y_batch)
        return loss

    for i in prange(epoch):
        epoch_loss = 0
        
        # Iterate through the dataset in batches
        for a in prange(0, len(X_train), batch_size):
            X_batch = X_train[a:a+batch_size]
            Y_batch = Y_train[a:a+batch_size]

            optimizer.zero_grad()

            # Use vmap to apply the model and compute the loss over the batch
            losses = vmap(compute_loss)(X_batch, Y_batch)

            # Sum the losses across the batch and backpropagate
            loss = losses.sum()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss = epoch_loss / (len(X_train) / batch_size)
        loss_calc.append(epoch_loss)

        if i % 100 == 0:
            print("Epoch number", i)
            print(epoch_loss)

    return loss_calc


In [15]:
%%time
model_fn(X_train, Y_train, model, optimizer, loss_fn)

Epoch number 0
67.13478820825674
Epoch number 100
65.7927101222376
Epoch number 200
64.95917906146649
Epoch number 300
64.21829972259373
Epoch number 400
63.53047009665659
Epoch number 500
62.91081402702394
Epoch number 600
62.35267247111335
Epoch number 700
61.84785767597242
Epoch number 800
61.366534558244865
Epoch number 900
60.92561490928563
CPU times: user 12.8 s, sys: 37.9 ms, total: 12.9 s
Wall time: 3.22 s


[67.13478820825674,
 67.08692819492657,
 67.07257216984063,
 67.02460034792031,
 66.98364773076666,
 66.9856553381069,
 66.95428659709788,
 66.9084114336073,
 66.9070598359598,
 66.87854567709795,
 66.85910879262704,
 66.85130791718575,
 66.8461993530174,
 66.8217953966648,
 66.78437316048203,
 66.77406401268615,
 66.7571542002443,
 66.7614365681933,
 66.73751645609372,
 66.70712430869969,
 66.6996647957682,
 66.69316802406,
 66.6805791683726,
 66.66213668773459,
 66.62864259405588,
 66.62698624379095,
 66.61482384387665,
 66.59221278279291,
 66.5856586226048,
 66.56764754502264,
 66.54958996298262,
 66.54285869007204,
 66.52611411687407,
 66.50943979509114,
 66.50640018456912,
 66.50132724083656,
 66.48298172694047,
 66.46730972464283,
 66.46759270921618,
 66.43913890720387,
 66.42386367698283,
 66.42012254176677,
 66.39642604684752,
 66.40587337798824,
 66.38553114077396,
 66.38470989183735,
 66.35504400555112,
 66.35478278902188,
 66.35287313788011,
 66.33726940808444,
 66.306768420